In [1]:
import utils.message_constructor as mc
import utils.vlm_utils as vu
import torch
import json

/home/pengzy/mambaforge/envs/qwen2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-07-24 18:24:47,645] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio
collect2: error: ld returned 1 exit status


 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.


/home/pengzy/mambaforge/envs/qwen2/lib/python3.11/site-packages/deepspeed/runtime/zero/linear.py:47: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @autocast_custom_fwd
/home/pengzy/mambaforge/envs/qwen2/lib/python3.11/site-packages/deepspeed/runtime/zero/linear.py:66: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @autocast_custom_bwd


In [2]:
json_path = r"./sln/VQA_SA/VQA-SA-question.json"
llm_base = r"./sln/qwen2.5_lm_7B"
device = "cuda:1"

In [8]:
with open(json_path, "r", encoding="utf8") as reader:
    data = json.load(reader)
if_dict = dict()
key_seq = list()
for e in data:
    if e["image_path"] not in if_dict:
        if_dict[e["image_path"]] = list()
        key_seq.append(e["image_path"])
    if_dict[e["image_path"]].append(e["question"])

In [4]:
vu.ensure_vl_base(
    base_path=llm_base,
    device=device,
    torch_dtype=torch.bfloat16,
)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.45s/it]


vlm based on <class 'transformers.models.qwen2.modeling_qwen2.Qwen2ForCausalLM'> is loaded


In [5]:
llm_model = vu.vl_model
llm_processor = vu.vl_processor

def qwen25_infer(
        messages:mc.Message,
        max_new_tokens:int=vu.DEFAULT_MAX_NEW_TOKENS
    ) -> str:
    text = llm_processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = llm_processor(text=[text], return_tensors="pt")
    inputs = inputs.to(llm_model.device)
    generated_ids = llm_model.generate(**inputs, max_new_tokens=max_new_tokens)
    generated_ids_trimmed = [out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)]
    output_text = llm_processor.batch_decode(generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    return "".join(output_text)

In [67]:
%load_ext autoreload
%autoreload 2
import utils.prompts as sp

chosen = key_seq[1567]
print(chosen)

def input_construct(ques_list:list) -> str:
    ques_list = [f"{(i + 1)}、{ques_list[i]}" for i in range(len(ques_list))]
    return "\n".join(ques_list)

user_utter = input_construct(if_dict[chosen])

message = mc.Message().add_role_block("system", sp.PARSER_SYSTEM)\
                .add_role_block("user", user_utter)

print(user_utter)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
images\26_2_104_0000029_240921.jpg
1、身穿黄色衣服的男人位于穿花斑吊带女人的什么位置？请从前方、左方、右方、后方中选择一个回答
2、位于近处桌上正放的杯子和手提包哪个所处的位置更高？请从杯子和手提包中选择一个回答
3、位于近处桌上正放的杯子和手提包哪个离图片中间区域的风扇更远？请从杯子和手提包中选择一个回答


In [68]:
output = qwen25_infer(message, 2048)
print(output)

```json
[
    {
        "reference": null,
        "observe": ["身穿黄色衣服的男人", "穿花斑吊带的女人"],
        "options": ["前方", "左方", "右方", "后方"]
    },
    {
        "reference": null,
        "observe": ["杯子", "手提包"],
        "options": ["杯子", "手提包"]
    },
    {
        "reference": null,
        "observe": ["杯子", "手提包"],
        "options": ["杯子", "手提包"]
    }
]
```
